#  Sales Data Analytics with SQL

This project simulates an e-commerce database and uses **SQL queries in Python** (with SQLite) to analyze:

- Customer spending
- Product sales
- Monthly orders
- Average order value

### 🔧 Technologies:
- Python
- SQLite (in-memory)
- Pandas


In [7]:
import sqlite3
import pandas as pd

conn = sqlite3.connect(':memory:')
cursor = conn.cursor()
cursor.execute('''
CREATE TABLE customers (
    id INTEGER PRIMARY KEY,
    name TEXT,
    email TEXT
)
''')











In [8]:
cursor.execute('''
CREATE TABLE orders (
    id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    order_date TEXT,
    total REAL,
    FOREIGN KEY (customer_id) REFERENCES customers(id)
)
''')

cursor.execute('''
CREATE TABLE order_items (
    id INTEGER PRIMARY KEY,
    order_id INTEGER,
    product TEXT,
    quantity INTEGER,
    price REAL,
    FOREIGN KEY (order_id) REFERENCES orders(id)
)
''')


In [9]:
cursor.executemany('INSERT INTO customers (id, name, email) VALUES (?, ?, ?)', [
    (1, 'Alice', 'alice@example.com'),
    (2, 'Bob', 'bob@example.com'),
    (3, 'Charlie', 'charlie@example.com')
])

cursor.executemany('INSERT INTO orders (id, customer_id, order_date, total) VALUES (?, ?, ?, ?)', [
    (1, 1, '2025-06-01', 150.0),
    (2, 2, '2025-06-02', 90.0),
    (3, 1, '2025-06-10', 200.0)
])

cursor.executemany('INSERT INTO order_items (id, order_id, product, quantity, price) VALUES (?, ?, ?, ?, ?)', [
    (1, 1, 'Keyboard', 1, 50.0),
    (2, 1, 'Mouse', 2, 50.0),
    (3, 2, 'Monitor', 1, 90.0),
    (4, 3, 'Laptop', 1, 200.0)
])



In [10]:
#Topselling products

query = '''
SELECT c.name, SUM(o.total) AS total_spent
FROM customers c
JOIN orders o ON c.id = o.customer_id
GROUP BY c.name
ORDER BY total_spent DESC
'''
pd.read_sql_query(query, conn)



,name,total_spent
0,Alice,350.0
1,Bob,90.0


In [11]:
#Orders in June

query = '''
SELECT id, customer_id, order_date, total
FROM orders
WHERE strftime('%m', order_date) = '06'
'''
pd.read_sql_query(query, conn)



,id,customer_id,order_date,total
0,1,1,2025-06-01,150.0
1,2,2,2025-06-02,90.0
2,3,1,2025-06-10,200.0


In [12]:
#D.avg order value per customer if>$100

query = '''
SELECT c.name, AVG(o.total) as avg_order
FROM customers c
JOIN orders o ON c.id = o.customer_id
GROUP BY c.name
HAVING avg_order > 100
'''
pd.read_sql_query(query, conn)

,name,avg_order
0,Alice,175.0
